In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################


username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 31799
db = "AAC"
col = "animals"
# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, db, col)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard - Chapman'))),
    html.Center(html.Img(src='data:image/png;base64, {}'.format(encoded_image.decode()),
                       style={'height': '100px', 'margin': '10px'})),
    html.Hr(),
    html.Div([
        html.H3('Filter Options', style={'text-align': 'center', 'color': '#2E86AB'}),
        html.Div([
            dcc.RadioItems(
                id='filter_type',
                options=[
                    {'label': 'Water Rescue', 'value': 'water'},
                    {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                    {'label': 'Disaster/Individual Tracking', 'value': 'disaster'},
                    {'label': 'Reset (Show All)', 'value': 'reset'}
                ],
                value='reset',
                labelStyle={'display': 'inline-block', 'margin': '10px'},
                style={'text-align': 'center'}
            )
        ])
    ],   


    style = {'margin': '20px', 'padding': '20px', 'backgroundColor': '#F8F9FA', 'borderRadius': '10px'}),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable="single",
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10,
                         style_cell={'textAlign': 'left'},
                         style_header={'backgroundColor': '#2E86AB', 'color': 'white', 'fontWeight': 'bold'},
                         style_data_conditional=[
                             {
                                 'if':{'row_index':'odd'},
                                 'backgroundColor': 'rgb(248, 248, 248)'
                             }
                         ]


                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
def get_rescue_filter(filter_type):
    
    if filter_type == 'water':
        return {
            "$and": [
                {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                {"sex_upon_outcome": "Intact Female"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'mountain':
        return {
            "$and": [
                {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'disaster':
        return {
            "$and": [
                {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                                  "Bloodhound", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
            ]
        }
    else:
        return {}

    
@app.callback(Output('datatable-id','data'),
              [Input('filter_type', 'value')])
def update_dashboard(filter_type):
    query_filter = get_rescue_filter(filter_type)
    filtered_data = db.read(query_filter)
    filtered_df = pd.DataFrame.from_records(filtered_data)
    if not filtered_df.empty and '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [html.P("No data to display")]
    dff = pd.DataFrame.from_dict(viewData)
    breed_counts = dff['breed'].value_counts().head(10)
    fig = px.pie(
        values=breed_counts.values,
        names=breed_counts.index,
        title='Top 10 Breed Distributions'
    )
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(showlegend=True, height=400)
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else: 
        row = min(index[0], len(dff) -1)
        
    # Austin TX is at [30.75,-97.48]
    if len(dff.columns)>14 and len(dff) >row:
        lat=dff.iloc[row][13]
        lon=dff.iloc[row][14]
        try:
            lat = float(lat)
            lon = float(lon)
        except:
            lat, lon = 30.75, -97.48
    
    if not lat and lon:
        lat, lon = 30.75, -97.48
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat,lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[lat,lon], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Connected to MongoDB
Dash app running on http://127.0.0.1:11366/
